# NREL Wave Hindacast Data Access Example

#### This notebook demonstrates basic usage of the National Renewable Energy Laboratory (NREL) West Coast Wave Hindcast dataset. The data is provided from Amazon Web Services using the HDF Group's Highly Scalable Data Service (HSDS).

#### For this example to work, the packages necessary can be installed via pip:

pip install -r requirements.txt

#### Next you'll need to configure h5pyd to access the data on HSDS:

hsconfigure

#### and enter at the prompt:

hs_endpoint = https://developer.nrel.gov/api/hsds   
hs_username = None   
hs_password = None   
hs_api_key = 3K3JQbjZmWctY0xmIfSYvYgtIcM3CN0cb1Y2w9bf    

#### The example API key here is for demonstation and is rate-limited per IP. To get your own API key, visit https://developer.nrel.gov/signup/

#### You can also add the above contents to a configuration file at ~/.hscfg

#### Finally, you can use Jupyter Notebook or Lab to view the example notebooks depending on your preference

For example, to start the jupyter notebook server on your machine, type the following at a command prompt in the folder containing these files:

    jupyter notebook

This should start the server and open a browser window that contains these files, then open *this* ipython notebook, and get started!


In [1]:
import h5pyd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Basic Usage

The Wave Hindcast Dataset is provided in annual .h5 files and currently spans the Western Coastal Region of the Lower 48 from 1979-2010.

Each year can be accessed from /nrel/us-wave/US_Wave_${year}.h5

To open the desired year of Wave Hindcast data server endpoint, username, password is found via a config file

In [3]:
waves = h5pyd.File(f'/nrel/us-wave/US_Wave_1990.h5','r')

In [4]:
list(waves) # list of base attributes of the dataset

['coordinates',
 'directionality_coefficient',
 'energy_period',
 'maximum_energy_direction',
 'mean_absolute_period',
 'mean_wave_direction',
 'mean_zero-crossing_period',
 'meta',
 'omni-directional_wave_power',
 'peak_period',
 'significant_water_height',
 'spectral_width',
 'time_index',
 'water_depth']

## Datasets

Each dataset quantity:

- directionality_coefficient
- energy_period
- maximum_energy_direction
- mean_absolute_period
- mean_zero-crossing_period
- omni-directional_wave_power
- peak_period
- significant_wave_height
- spectral_width
- water_depth

is structured as ('time_index','coordinate')


In [5]:
# Shapes of datasets
waves['significant_water_height'].shape # (time_index, coordinates)

(2920, 699904)

In [6]:
# List the available dataset variable names within the dataset
print(list(waves['coordinates'].attrs))

['description', 'dimensions', 'src_name', 'units']


In [7]:
# Locational information is stored in either 'meta' or 'coordinates'

meta = pd.DataFrame(waves['meta'][:])
meta.head()

,water_depth,latitude,longitude,distance_to_shore,timezone,jurisdiction
0,6.718400,48.864101,-125.386002,993.098145,-8,b'Canada'
1,28.699100,48.841801,-125.445999,5096.523438,-8,b'Canada'
2,49.457100,48.819302,-125.503998,6850.421387,-8,b'Canada'
3,72.204498,48.797199,-125.563004,10784.603516,-8,b'Canada'
4,92.086197,48.773998,-125.622002,15433.410156,-8,b'Canada'


# Basic Usage

The following examples illustrate basic spatial and timeseries based slicing techniques for the West Coast Wave Hindcast dataset, along with simple statistical analysis. 

## Time Slicing

Basic description of timestamp extraction

In [8]:
# Extract datetime index for datasets (Saved as binary)

time_index = pd.to_datetime(waves['time_index'][:].astype(str))
time_index, time_index.inferred_freq # Temporal resolution is 3 hours

(DatetimeIndex(['1990-01-01 00:00:00+00:00', '1990-01-01 03:00:00+00:00',
                '1990-01-01 06:00:00+00:00', '1990-01-01 09:00:00+00:00',
                '1990-01-01 12:00:00+00:00', '1990-01-01 15:00:00+00:00',
                '1990-01-01 18:00:00+00:00', '1990-01-01 21:00:00+00:00',
                '1990-01-02 00:00:00+00:00', '1990-01-02 03:00:00+00:00',
                ...
                '1990-12-30 18:00:00+00:00', '1990-12-30 21:00:00+00:00',
                '1990-12-31 00:00:00+00:00', '1990-12-31 03:00:00+00:00',
                '1990-12-31 06:00:00+00:00', '1990-12-31 09:00:00+00:00',
                '1990-12-31 12:00:00+00:00', '1990-12-31 15:00:00+00:00',
                '1990-12-31 18:00:00+00:00', '1990-12-31 21:00:00+00:00'],
               dtype='datetime64[ns, UTC]', length=2920, freq=None),
 '3H')

In [9]:
Aug_7th_midnight = time_index.get_loc('1990-08-07 00:00:00') #precise data returns single index
Aug = time_index.get_loc('1990-08') #Inprecise will return a range a values

In [10]:
print(f'August 7th midnight: {time_index[Aug_7th_midnight]}')
print(f'Month of August: {time_index[Aug]}')

August 7th midnight: 1990-08-07 00:00:00+00:00
Month of August: DatetimeIndex(['1990-08-01 00:00:00+00:00', '1990-08-01 03:00:00+00:00',
               '1990-08-01 06:00:00+00:00', '1990-08-01 09:00:00+00:00',
               '1990-08-01 12:00:00+00:00', '1990-08-01 15:00:00+00:00',
               '1990-08-01 18:00:00+00:00', '1990-08-01 21:00:00+00:00',
               '1990-08-02 00:00:00+00:00', '1990-08-02 03:00:00+00:00',
               ...
               '1990-08-30 18:00:00+00:00', '1990-08-30 21:00:00+00:00',
               '1990-08-31 00:00:00+00:00', '1990-08-31 03:00:00+00:00',
               '1990-08-31 06:00:00+00:00', '1990-08-31 09:00:00+00:00',
               '1990-08-31 12:00:00+00:00', '1990-08-31 15:00:00+00:00',
               '1990-08-31 18:00:00+00:00', '1990-08-31 21:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=248, freq=None)


## Mapping Data

Basic methods to quickly plot data from the Hindcast spatially and temporally

### Spatial Plots

Coordinate pair indices are used to slice the datasets, and are accessed through the 'coordinates' attribute

In [11]:
coord_slice = 100

print(f'Available attributes: {dict(waves["coordinates"].attrs)}')
coords = waves['coordinates'][::coord_slice]
print(f'Coordinate pairs: {coords}')

Available attributes: {'description': '(latitude, longitude) using Datum: NAD83', 'dimensions': array([array('position', dtype='<U8')], dtype=object), 'src_name': '(Xp, Yp)', 'units': '(deg, deg)'}
Coordinate pairs: [[  48.8641 -125.386 ]
 [  46.1511 -129.577 ]
 [  41.74   -129.538 ]
 ...
 [  37.4535 -122.072 ]
 [  37.4517 -122.055 ]
 [  37.4617 -122.027 ]]


In [12]:
# Building a scatter plot of significant wave height values

df = pd.DataFrame() # Combine data with coordinates in a DataFrame
df['longitude'] = coords[:, 1]
df['latitude'] = coords[:, 0]
df['Hsig'] = waves['significant_water_height'][Aug_7th_midnight, ::coord_slice] 

In [19]:
%matplotlib widget

df['Hsig'][df['Hsig']<0] = np.nan # There are negative values here Levi
df.plot.scatter(x='longitude', y='latitude', c='Hsig',
                colormap='viridis',
                title=str(time_index[Aug_7th_midnight]))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Timeseries Plots

Extracting a timeseries usually requires us to find a location of interest.      
This can be done easially with KDTree, a function within scipy.spatial.   


In [17]:
from scipy.spatial import cKDTree

tree = cKDTree(coords)
def nearest_site(tree, lat_coord, lon_coord):
    lat_lon = np.array([lat_coord, lon_coord])
    dist, pos = tree.query(lat_lon)
    return pos

lat_coord, lon_coord = 39.6122, -125.318
site_idx = nearest_site(tree, lat_coord, lon_coord)

print(f'Site index: {site_idx}')
print(f'Coordinates Searched: {lat_coord, lon_coord}')
print(f'Coordinates of Nearest Point: {coords[site_idx]}')

Site index: 2567
Coordinates Searched: (39.6122, -125.318)
Coordinates of Nearest Point: [  39.546 -125.174]


In [18]:
# Now we can extract data at that location over a specified time range

var = 'peak_period'

dt = pd.DataFrame(waves[var][Aug,site_idx],index=time_index[Aug],columns=[var])
dt.plot(y=var,title=f'Time sliced {var} Timeseries')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Basic Statistics and Grouping

Here is an example of grouping data to create quick plots of statistics

In [20]:
Vars =['significant_water_height','peak_period'] 
variables = np.array([waves[Vars[0]][Aug,site_idx], waves[Vars[1]][Aug,site_idx]])

ds = pd.DataFrame(variables.T,
                    index=time_index[Aug],
                    columns=Vars
                )
group = ds.groupby('peak_period')

In [21]:
# Plot the number of occurances of sea-states within peak period bins

group.count().plot(kind='bar')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
# plot the mean Hsig of the sea-states within peak period bins along with
# the variance of those waveheights

group.mean().plot(kind='bar',yerr=group.var())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …